In [17]:
import pandas as pd
BRCA_FR_SUPERFILTRATO_WG=pd.read_csv('/content/BRCA_FR_AF05_CLINSIG_NA_FILTRO_sample_inputmatrix.txt',sep='\t')

In [18]:
BRCA_FR_SUPERFILTRATO_WG

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
1,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
2,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
3,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
4,BRCA-FR,SA596039,SA596039,GRCh37,SNP,1,202468579,202468579,C,A,SOMATIC
...,...,...,...,...,...,...,...,...,...,...,...
420794,BRCA-FR,SA595120,SA595120,GRCh37,SNP,8,15395130,15395130,G,C,SOMATIC
420795,BRCA-FR,SA595120,SA595120,GRCh37,SNP,8,15395130,15395130,G,C,SOMATIC
420796,BRCA-FR,SA595120,SA595120,GRCh37,SNP,8,15395130,15395130,G,C,SOMATIC
420797,BRCA-FR,SA595120,SA595120,GRCh37,SNP,6,42376519,42376519,C,T,SOMATIC


In [19]:
ESONI=pd.read_csv('/content/drive/MyDrive/ESONI.tsv',sep='\t')

In [20]:
BRCA_FR_SUPERFILTRATO_WG['chrom']='chr'+ BRCA_FR_SUPERFILTRATO_WG['chrom'].astype(str)

In [21]:
ESONI['chrom']=ESONI['chrom'].astype(str)

In [22]:
ESONI['exonStarts']=ESONI['exonStarts'].astype(int)

In [23]:
ESONI['exonEnds']=ESONI['exonEnds'].astype(int)

In [24]:
BRCA_FR_SUPERFILTRATO_WG['pos_start']=BRCA_FR_SUPERFILTRATO_WG['pos_start'].astype(int)
BRCA_FR_SUPERFILTRATO_WG['pos_end']=BRCA_FR_SUPERFILTRATO_WG['pos_end'].astype(int)

In [25]:
n=int(0.2*(ESONI.shape[0]))

In [26]:
ESONI=ESONI.sample(n)

In [27]:
ESONI

,chrom,exonStarts,exonEnds,name2
169083,chr11,102207639,102210134,BIRC3
38203,chr2,238949928,238951423,UBE2F
234914,chr17,63532437,63532671,AXIN2
67304,chr4,79461684,79465423,FRAS1
105583,chr7,122765622,122765729,SLC13A1
...,...,...,...,...
81885,chr5,56155541,56155742,MAP3K1
269324,chr22,20907431,20907461,MED15
55042,chr3,167272437,167272586,WDR49
189026,chr13,111806012,111806093,ARHGEF7


In [28]:
import pandas as pd
import numpy as np
from multiprocessing import Pool

# Definizione della funzione per il confronto dei blocchi
def compare_blocks(block_BRCA, block_ESONI):
    # Resto del codice rimane invariato
    merged_block = block_BRCA.merge(block_ESONI, on='chrom', how='inner')
    filtered_block = merged_block[
        (merged_block['pos_start'] >= merged_block['exonStarts']) &
        (merged_block['pos_end'] <= merged_block['exonEnds'])
    ]
    return filtered_block[['Project', 'Sample', 'ID', 'Genome', 'mut_type', 'chrom', 'pos_start',
       'pos_end', 'ref', 'alt', 'Type']]

# Dimensione dei blocchi
block_size = 1000

# Calcolo del numero di blocchi per ciascun DataFrame
num_blocks_BRCA = int(np.ceil(len(BRCA_FR_SUPERFILTRATO_WG) / block_size))
num_blocks_ESONI = int(np.ceil(len(ESONI) / block_size))

# Creazione della lista di indici per i blocchi
block_indices = []
for i in range(num_blocks_BRCA):
    start_idx_BRCA = i * block_size
    end_idx_BRCA = min((i + 1) * block_size, len(BRCA_FR_SUPERFILTRATO_WG))

    for j in range(num_blocks_ESONI):
        start_idx_ESONI = j * block_size
        end_idx_ESONI = min((j + 1) * block_size, len(ESONI))

        block_indices.append([start_idx_BRCA, end_idx_BRCA, start_idx_ESONI, end_idx_ESONI])

# Funzione wrapper per la chiamata a compare_blocks con gli indici
def compare_blocks_wrapper(indices):
    block_BRCA = BRCA_FR_SUPERFILTRATO_WG.iloc[indices[0]:indices[1]]
    block_ESONI = ESONI.iloc[indices[2]:indices[3]]

    return compare_blocks(block_BRCA, block_ESONI)

# Utilizzo di multiprocessing per eseguire operazioni parallele
with Pool() as pool:
    results = pool.map(compare_blocks_wrapper, block_indices)

# Combinazione dei risultati in un unico DataFrame
final_result = pd.concat(results, ignore_index=True)

In [29]:
final_result

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr17,56160983,56160983,C,T,SOMATIC
1,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr11,2716984,2716984,C,G,SOMATIC
2,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr8,110661094,110661094,G,A,SOMATIC
3,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr8,110661094,110661094,G,A,SOMATIC
4,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr8,110661094,110661094,G,A,SOMATIC
...,...,...,...,...,...,...,...,...,...,...,...
16477,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr3,48452417,48452417,G,C,SOMATIC
16478,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr3,48452417,48452417,G,C,SOMATIC
16479,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr3,48452417,48452417,G,C,SOMATIC
16480,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr3,48452417,48452417,G,C,SOMATIC


In [30]:
for i in range(final_result.shape[0]):
  final_result.loc[i,'chrom']= final_result.loc[i,'chrom'][3:]

In [31]:
final_result

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,BRCA-FR,SA596039,SA596039,GRCh37,SNP,17,56160983,56160983,C,T,SOMATIC
1,BRCA-FR,SA596039,SA596039,GRCh37,SNP,11,2716984,2716984,C,G,SOMATIC
2,BRCA-FR,SA596039,SA596039,GRCh37,SNP,8,110661094,110661094,G,A,SOMATIC
3,BRCA-FR,SA596039,SA596039,GRCh37,SNP,8,110661094,110661094,G,A,SOMATIC
4,BRCA-FR,SA596039,SA596039,GRCh37,SNP,8,110661094,110661094,G,A,SOMATIC
...,...,...,...,...,...,...,...,...,...,...,...
16477,BRCA-FR,SA595120,SA595120,GRCh37,SNP,3,48452417,48452417,G,C,SOMATIC
16478,BRCA-FR,SA595120,SA595120,GRCh37,SNP,3,48452417,48452417,G,C,SOMATIC
16479,BRCA-FR,SA595120,SA595120,GRCh37,SNP,3,48452417,48452417,G,C,SOMATIC
16480,BRCA-FR,SA595120,SA595120,GRCh37,SNP,3,48452417,48452417,G,C,SOMATIC


In [32]:
# Salva il DataFrame come file di testo separato da virgole (CSV)
final_result.to_csv('brca_fr_target_af05naclinsig.txt', sep='\t', index=False)  # Separa le colonne con il carattere di tabulazione ('\t')